In [3]:
from elasticsearch import Elasticsearch
import os
import re

In [5]:
res_path = "../res"
docs_path = os.path.join(res_path, "docs")

physical_health = os.path.join(docs_path, "Salud Fisica")
mental_health = os.path.join(docs_path, "Salud Mental")
wellness = os.path.join(docs_path, "Bienestar")


Text processing

In [6]:
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales
    text = re.sub(r'[\W_]+', ' ', text)
    return text

def read_and_preprocess(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return preprocess_text(content)

Elastic Search

In [7]:
es = Elasticsearch(["http://localhost:9200"])

def index_document(file_path, category):
    content = read_and_preprocess(file_path)
    doc_id = os.path.basename(file_path)
    es.index(index="health_index", id=doc_id, body={"category": category, "content": content})

def index_documents_from_directory(directory_path, category):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            index_document(file_path, category)

In [8]:
index_documents_from_directory(mental_health, "mental_health")
index_documents_from_directory(physical_health, "physical_health")
index_documents_from_directory(wellness, "general_wellness")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte

In [ ]:
def search_documents(category, query):
    body = {
        "query": {
            "bool": {
                "must": {
                    "match": {"content": query}
                },
                "filter": {
                    "term": {"category": category}
                }
            }
        }
    }
    results = es.search(index="health_index", body=body)
    return results['hits']['hits']

In [ ]:
results = search_documents("mental_health", "depression")
for result in results:
    print(result['_source']['content'])